In [1]:
import os
os.chdir("../")

In [2]:
import cv2
import mediapipe as mp
import numpy as np

from src.utils.config import load_config

In [3]:
_, local_config = load_config()

test_filename = "vidal_test.mp4"

cap = cv2.VideoCapture(
    local_config["PATH"]["raw_videos"]+test_filename
)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

if (cap.isOpened()== False): 
  print("Error opening video stream or file")

out = cv2.VideoWriter(
    local_config["PATH"]["raw_videos"]+"vidal_test_prediction.avi",
    cv2.VideoWriter_fourcc('M','J','P','G'),
    24,
    (frame_width,frame_height)
)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
BG_COLOR = (192, 192, 192)

In [4]:
with mp_pose.Pose(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5
) as pose:
    
    while (cap.isOpened()):
        
        ret, frame = cap.read()
        if ret == True:
            
            frame_result = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            
            annotated_frame = frame.copy()
            
            # Draw pose landmarks on the image.
            mp_drawing.draw_landmarks(
                annotated_frame,
                frame_result.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

            out.write(annotated_frame)
        else: break
            
            
cap.release()
out.release()        